In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [5]:
cg.get_parent(147640497232412734)

In [4]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [5]:
from pychunkedgraph.graph.edits import add_edge
from pychunkedgraph.graph.utils.context_managers import TimeIt

# with TimeIt("get_cross_chunk_edges"):
#     edges = cg.get_cross_chunk_edges([435828817064694903])
#     print(edges)

edges = np.array([[75512534450307707, 75582971913961503]], dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edge(cg, edges=edges))

get_atomic_cross_edges multiple 0.1023249626159668
get_cross_chunk_edges cross_edges_d 0.4097921848297119
2 [147570128488235009, 147640565951889409]
3 [219564225729658881, 219704963218014228]
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
[864691128455135254, 864691128455135255]
add_edge 2.303920269012451


In [3]:
cg.get_roots([75582903194485666, 75582834475009193])

array([864691138338302132, 864691138338302132], dtype=uint64)